In [15]:
import mlflow
import mlflow.sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import datasets

In [7]:
mlflow.get_tracking_uri()

'http://localhost:5000'

In [4]:
mlflow.set_tracking_uri("http://localhost:5000")

In [5]:
mlflow.get_tracking_uri()

'http://localhost:5000'

In [8]:
mlflow.set_experiment("iris") # creates an experiment if it doesn't exist

2024/01/17 09:29:54 INFO mlflow.tracking.fluent: Experiment with name 'iris' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/251967034232254676', creation_time=1705480194253, experiment_id='251967034232254676', last_update_time=1705480194253, lifecycle_stage='active', name='iris', tags={}>

In [9]:
iris = datasets.load_iris()
x = iris.data[:, 2:]
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=7)

In [16]:
import joblib

with mlflow.start_run(run_name="Iris KNN Experiment") as run:
    
    # Add parameters for tuning
    num_neighbors = 4
    mlflow.log_param("num_neighbors", num_neighbors)

    # Train the model
    knn = KNeighborsClassifier(n_neighbors=num_neighbors)
    knn.fit(X_train, y_train)
    predictions = knn.predict(X_test)

    # Save the model artifact using joblib.dump
    model_path = "knn-model.joblib"
    joblib.dump(knn, model_path)
    mlflow.log_artifact(model_path, "model")

    # Log model performance 
    mse = mean_squared_error(y_test, predictions)
    mlflow.log_metric("mse", mse)
    print("  MSE: %f" % mse)

    run_id = run.info.run_uuid
    experiment_id = run.info.experiment_id
    
    # End the MLflow run
    mlflow.end_run()

    # Print artifact URI and run ID
    print(mlflow.get_artifact_uri())
    print("Run ID: %s" % run_id)

  MSE: 0.133333
mlflow-artifacts:/251967034232254676/94e6289e5be54ac5bf3a8ab37869a8c1/artifacts
Run ID: dd0845f1ae3744f89c23a872c97871bd


In [17]:
model_path = "knn-model.joblib"

# Chargemenet du modèle
loaded_model = joblib.load(model_path)

predictions = loaded_model.predict(X_test)
